In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[11],9
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.5e-24
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[3, 3]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.132474                    1  0.283140
0.000750        2  0.010364                    2  0.097323
0.001052        3  0.006665                    3  0.050919
0.001476        4  0.001568                    4  0.013060
0.002070        5 -0.003415                    5 -0.016482
0.002904        6 -0.007743                    6 -0.030896
0.004074        7 -0.011661                    7 -0.038344
0.005714        8 -0.015487                    8 -0.038875
0.008015        9 -0.019382                    9 -0.040216
0.011243       10 -0.023514                   10 -0.040032
0.015771       11 -0.027995                   11 -0.041131
0.022122       12 -0.032999                   12 -0.043730
0.031031       13 -0.039068                   13 -0.047906
0.043528       14 -0.049244                   14 -0.057966
0.061057       15 -0.061178                   15 -0.068846
0.085645       16 -0.065256                   16 -0.067293
0.120136       17 -0.061071                   17 -0.052435
0.168516       18 -0.055987                   18 -0.034395
0.236378       19 -0.045017                   19 -0.015884
0.331549       20 -0.022706                   20  0.001499
0.465100       21  0.020035                   21  0.024742
0.652400       22  0.116627                   22  0.095379
0.915100       23  0.267705                   23  0.227541
1.283650       24  0.266036                   24  0.237705
1.800600       25  0.149019                   25  0.163414
2.525700       26  0.080983                   26  0.113222
3.542800       27  0.044815                   27  0.069920
4.969550       28  0.023997                   28  0.031078
6.970850       29  0.012753                   29  0.007462
9.778100       30  0.006386                   30  0.003013
13.715850      31  0.001971                   31  0.004941
19.239350      32 -0.000880                   32  0.003131
26.987250      33 -0.001438                   33  0.000495
37.855300      34 -0.001031                   34 -0.000955
53.100050      35 -0.001162                   35 -0.001453
73.887500      36 -0.001621                   36 -0.001774
97.662500      37 -0.001857                   37 -0.001905
121.437500     38 -0.001955                   38 -0.001949
145.212500     39 -0.002453                   39 -0.002502
168.987500     40 -0.003266                   40 -0.003417
192.762500     41 -0.003714                   41 -0.003897
216.537500     42 -0.003539                   42 -0.003763
240.312500     43 -0.003489                   43 -0.003795
264.087500     44 -0.003526                   44 -0.003932
287.862500     45 -0.003551                   45 -0.003982
311.637500     46 -0.003547                   46 -0.004033
335.412500     47 -0.003508                   47 -0.003907
359.187500     48 -0.003430                   48 -0.003796
382.962500     49 -0.003305                   49 -0.003584
406.737500     50 -0.003131                   50 -0.003296
430.512500     51 -0.002920                   51 -0.002936
454.287500     52 -0.002697                   52 -0.002651
478.062500     53 -0.002482                   53 -0.002230
501.837500     54 -0.002273                   54 -0.001915
525.612500     55 -0.002051                   55 -0.001583
549.387500     56 -0.001802                   56 -0.001238
573.162500     57 -0.001522                   57 -0.000878
596.937500     58 -0.001208                   58 -0.000498
620.712500     59 -0.000860                   59 -0.000095
644.487500     60 -0.000478                   60  0.000334
668.262500     61 -0.000057                   61  0.000831
692.037500     62  0.000407                   62  0.001185
715.812500     63  0.000926                   63  0.001841
739.587500     64  0.001518                   64  0.002518
763.362500     65  0.002215                   65  0.003171
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -5.482672  0.000000 -5.482672 -5.396197  7.693315e-09   
0.000624    2     -5.482677  0.000014 -5.482662 -5.396204  2.764611e-05   
0.000876    3     -5.482679  0.000017 -5.482662 -5.396207  3.353038e-05   
0.001229    4     -5.482681  0.000019 -5.482662 -5.396211  3.961945e-05   
0.001723    5     -5.482685  0.000023 -5.482662 -5.396216  4.556551e-05   
0.002417    6     -5.482690  0.000028 -5.482662 -5.396223  5.112967e-05   
0.003391    7     -5.482696  0.000033 -5.482663 -5.396232  5.673524e-05   
0.004757    8     -5.482705  0.000040 -5.482665 -5.396245  6.299568e-05   
0.006672    9     -5.482716  0.000048 -5.482668 -5.396261  7.097363e-05   
0.009359    10    -5.482733  0.000058 -5.482674 -5.396285  8.138060e-05   
0.013128    11    -5.482755  0.000070 -5.482685 -5.396317  9.563460e-05   
0.018415    12    -5.482787  0.000085 -5.482702 -5.396362  1.150731e-04   
0.025830    13    -5.482834  0.000102 -5.482732 -5.396427  1.415627e-04   
0.036232    14    -5.482902  0.000122 -5.482780 -5.396522  1.776179e-04   
0.050823    15    -5.483011  0.000146 -5.482865 -5.396670  2.258869e-04   
0.071291    16    -5.483198  0.000185 -5.483013 -5.396912  3.009692e-04   
0.100000    17    -5.483512  0.000277 -5.483235 -5.397292  4.519952e-04   
0.140271    18    -5.484008  0.000482 -5.483526 -5.397857  7.670040e-04   
0.196760    19    -5.484781  0.000880 -5.483901 -5.398685  1.364338e-03   
0.275997    20    -5.485958  0.001634 -5.484324 -5.399880  2.410625e-03   
0.387100    21    -5.487680  0.003057 -5.484622 -5.401566  4.115831e-03   
0.543100    22    -5.489987  0.005734 -5.484252 -5.403797  6.804237e-03   
0.761700    23    -5.491951  0.010719 -5.481232 -5.405672  1.114974e-02   
1.068500    24    -5.490177  0.018674 -5.471503 -5.403838  1.758562e-02   
1.498800    25    -5.483984  0.026041 -5.457943 -5.397986  2.385101e-02   
2.102400    26    -5.476854  0.029567 -5.447288 -5.391219  2.876934e-02   
2.949000    27    -5.470008  0.030842 -5.439166 -5.384502  3.340812e-02   
4.136600    28    -5.463736  0.030874 -5.432862 -5.378013  3.675634e-02   
5.802500    29    -5.458185  0.030059 -5.428126 -5.371930  3.680682e-02   
8.139200    30    -5.453277  0.028681 -5.424596 -5.366195  3.313780e-02   
11.417000   31    -5.448881  0.026764 -5.422117 -5.360740  2.885319e-02   
16.014700   32    -5.445177  0.024134 -5.421043 -5.355930  2.673489e-02   
22.464000   33    -5.442687  0.020972 -5.421716 -5.352443  2.563925e-02   
31.510500   34    -5.441471  0.018215 -5.423256 -5.350257  2.398456e-02   
44.200100   35    -5.441209  0.016402 -5.424807 -5.348952  2.124343e-02   
62.000000   36    -5.442347  0.015091 -5.427256 -5.348994  1.822244e-02   
85.775000   37    -5.445970  0.014148 -5.431822 -5.351976  1.620697e-02   
109.550000  38    -5.451056  0.014005 -5.437051 -5.356902  1.576572e-02   
133.325000  39    -5.457069  0.014511 -5.442558 -5.362938  1.631151e-02   
157.100000  40    -5.464547  0.015081 -5.449465 -5.370646  1.697183e-02   
180.875000  41    -5.474803  0.016141 -5.458662 -5.381607  1.830651e-02   
204.650000  42    -5.488307  0.019187 -5.469121 -5.396279  2.200320e-02   
228.425000  43    -5.504178  0.025090 -5.479088 -5.413612  2.873718e-02   
252.200000  44    -5.522097  0.033184 -5.488913 -5.433124  3.756060e-02   
275.975000  45    -5.542040  0.043197 -5.498843 -5.454822  4.818327e-02   
299.750000  46    -5.563994  0.055150 -5.508843 -5.478630  6.077410e-02   
323.525000  47    -5.587932  0.069099 -5.518833 -5.504623  7.540839e-02   
347.300000  48    -5.613812  0.085098 -5.528714 -5.532621  9.240040e-02   
371.075000  49    -5.641563  0.103189 -5.538373 -5.562629  1.117179e-01   
394.850000  50    -5.671080  0.123397 -5.547682 -5.594532  1.335249e-01   
418.625000  51    -5.702203  0.145702 -5.556501 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')